In [1]:
# !unzip C3W2A2.zip

Archive:  C3W2A2.zip
  inflating: C3_W2_RecSysNN_Assignment.ipynb  
  inflating: public_tests.py         
  inflating: recsysNN_utils.py       
   creating: data/
  inflating: data/content_item_train.csv  
  inflating: data/content_item_train_header.txt  
  inflating: data/content_item_vecs.csv  
  inflating: data/content_movie_list.csv  
  inflating: data/content_user_to_genre.pickle  
  inflating: data/content_user_train.csv  
  inflating: data/content_user_train_header.txt  
  inflating: data/content_y_train.csv  
   creating: images/
  inflating: images/ColabFilterLearn.PNG  
  inflating: images/ColabFilterUse.PNG  
  inflating: images/distmatrix.PNG   
  inflating: images/film_award.png   
  inflating: images/film_filter.png  
  inflating: images/film_man_action.png  
  inflating: images/film_movie_camera.png  
  inflating: images/film_rating.png  
  inflating: images/film_reel.png    
  inflating: images/film_strip_vertical.png  
  inflating: images/movie_camera.png  
  inflating

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [5]:
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
5.487468450000023
GPU (s):
0.10164805699997714
GPU speedup over CPU: 53x


In [4]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

In [6]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True  # applies the standard scalar to data if true
print("shape of the user training data:", user_train.shape)
print("shape of the item training data:", item_train.shape)
print(f"Number of training vectors: {len(item_train)}")

shape of the user training data: (58187, 17)
shape of the item training data: (58187, 17)
Number of training vectors: 58187


In [7]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9
2,16,4.1,3.9,5.0,0.0,0.0,4.0,4.2,4.0,4.0,0.0,3.0,4.0,0.0,4.2,3.9


In [8]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6874,2003,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8798,2004,3.8,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [9]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  4.  4.  3.5 3.5]


In [10]:
# scale training data
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print(np.allclose(item_train_save, scalerItem.inverse_transform(item_train)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_train)))

True
True


In [11]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test  data shape: {item_test.shape}")

movie/item training data shape: (46549, 17)
movie/item test  data shape: (11638, 17)


In [12]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,0.6,0.7,0.6,0.6,0.7,0.7,0.5,0.7,0.2,0.3,0.3,0.5,0.5,0.8,0.5
0,0,1.6,1.5,1.7,0.9,1.0,1.4,0.8,-1.2,1.2,1.2,1.6,0.9,1.4,1.2,1.0
0,0,0.8,0.6,0.7,0.5,0.6,0.6,0.3,-1.2,0.7,0.8,0.9,0.6,0.2,0.6,0.6
1,0,-0.1,0.2,-0.1,0.3,0.7,0.3,0.2,1.0,-0.5,-0.7,-2.1,0.5,0.7,0.3,0.0
-1,0,-1.3,-0.8,-0.8,0.1,-0.1,-1.1,-0.9,-1.2,-1.5,-0.6,-0.5,-0.6,-0.9,-0.4,-0.9


In [13]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(46549, 1) (11638, 1)


In [14]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(46549, 1) (11638, 1)


In [15]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41376     ['input_2[0][0]']             
                                                                                              

In [16]:
# Public tests
from public_tests import *
test_tower(user_NN)
test_tower(item_NN)

All tests passed!
All tests passed!


In [17]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [22]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], ynorm_train, epochs=30)

Epoch 1/30
1455/1455 [==============================] - 10s 4ms/step - loss: 0.1249
Epoch 2/30
1455/1455 [==============================] - 7s 4ms/step - loss: 0.1183
Epoch 3/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1169
Epoch 4/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1150
Epoch 5/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1138
Epoch 6/30
1455/1455 [==============================] - 7s 5ms/step - loss: 0.1125
Epoch 7/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1117
Epoch 8/30
1455/1455 [==============================] - 7s 5ms/step - loss: 0.1111
Epoch 9/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1101
Epoch 10/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1091
Epoch 11/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1075
Epoch 12/30
1455/1455 [==============================] - 6s 4ms/step - loss: 0.1070


In [23]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)

364/364 [==============================] - 1s 2ms/step - loss: 0.1046


0.10459551215171814

In [44]:
new_user_id = 5000
new_rating_ave = 1.0
new_action = 1.0
new_adventure = 1
new_animation = 5
new_childrens = 5
new_comedy = 1
new_crime = 1
new_documentary = 1
new_drama = 1
new_fantasy = 1
new_horror = 1
new_mystery = 1
new_romance = 1
new_scifi = 1
new_thriller = 1
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [45]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs,  item_vecs, model, u_s, i_s,
                                                                       scaler, scalerUser, scalerItem, scaledata=scaledata)

print_pred_movies(sorted_ypu, sorted_user, sorted_items, movie_dict, maxcount = 10)

59/59 [==============================] - 0s 2ms/step


y_p,movie id,rating ave,title,genres
3.60319,5618,4.15517,Spirited Away (Sen to Chihiro no kamikakushi) (2001),Adventure|Animation|Fantasy
3.5658,4993,4.10606,"Lord of the Rings: The Fellowship of the Ring, The (2001)",Adventure|Fantasy
3.52338,5952,4.02128,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
3.4652,5072,3.96154,Metropolis (2001),Animation|Sci-Fi
3.446,6377,3.96099,Finding Nemo (2003),Adventure|Animation|Children|Comedy
3.44549,7090,3.92105,Hero (Ying xiong) (2002),Action|Adventure|Drama


<table>
<thead>
<tr><th style="text-align: right;">    y_p</th><th style="text-align: right;">  movie id</th><th style="text-align: right;">  rating ave</th><th>title                                                    </th><th>genres                             </th></tr>
</thead>
<tbody>
<tr><td style="text-align: right;">3.60319</td><td style="text-align: right;">      5618</td><td style="text-align: right;">     4.15517</td><td>Spirited Away (Sen to Chihiro no kamikakushi) (2001)     </td><td>Adventure|Animation|Fantasy        </td></tr>
<tr><td style="text-align: right;">3.5658 </td><td style="text-align: right;">      4993</td><td style="text-align: right;">     4.10606</td><td>Lord of the Rings: The Fellowship of the Ring, The (2001)</td><td>Adventure|Fantasy                  </td></tr>
<tr><td style="text-align: right;">3.52338</td><td style="text-align: right;">      5952</td><td style="text-align: right;">     4.02128</td><td>Lord of the Rings: The Two Towers, The (2002)            </td><td>Adventure|Fantasy                  </td></tr>
<tr><td style="text-align: right;">3.4652 </td><td style="text-align: right;">      5072</td><td style="text-align: right;">     3.96154</td><td>Metropolis (2001)                                        </td><td>Animation|Sci-Fi                   </td></tr>
<tr><td style="text-align: right;">3.446  </td><td style="text-align: right;">      6377</td><td style="text-align: right;">     3.96099</td><td>Finding Nemo (2003)                                      </td><td>Adventure|Animation|Children|Comedy</td></tr>
<tr><td style="text-align: right;">3.44549</td><td style="text-align: right;">      7090</td><td style="text-align: right;">     3.92105</td><td>Hero (Ying xiong) (2002)                                 </td><td>Action|Adventure|Drama             </td></tr>
</tbody>
</table>

In [41]:
uid =  36
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, scalerUser.inverse_transform(user_train), item_vecs, user_to_genre)

# scale the vectors and make predictions for all movies. Return results sorted by rating.
sorted_index, sorted_ypu, sorted_items, sorted_user = predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler,
                                                                      scalerUser, scalerItem, scaledata=scaledata)
sorted_y = y_vecs[sorted_index]

#print sorted predictions
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, item_features, ivs, uvs, movie_dict, maxcount = 10)


59/59 [==============================] - 0s 2ms/step


y_p,y,user,user genre ave,movie rating ave,title,genres
3.4,3.0,36,3.00,2.86,"Time Machine, The (2002)",Adventure
3.4,3.0,36,3.00,2.86,"Time Machine, The (2002)",Action
3.3,3.0,36,3.00,2.86,"Time Machine, The (2002)",Sci-Fi
1.9,2.0,36,1.75,3.52,Gangs of New York (2002),Crime
1.9,1.5,36,1.75,3.52,Road to Perdition (2002),Crime
1.9,1.0,36,1.50,4.00,"Beautiful Mind, A (2001)",Drama
1.5,1.0,36,1.00,4.00,"Beautiful Mind, A (2001)",Romance
1.5,2.0,36,1.50,3.52,Gangs of New York (2002),Drama
1.5,1.5,36,1.50,3.52,Road to Perdition (2002),Drama


<table>
<thead>
<tr><th style="text-align: right;">  y_p</th><th style="text-align: right;">  y</th><th style="text-align: right;">  user</th><th style="text-align: right;">  user genre ave</th><th style="text-align: right;">  movie rating ave</th><th>title                   </th><th>genres   </th></tr>
</thead>
<tbody>
<tr><td style="text-align: right;">  3.4</td><td style="text-align: right;">3.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            3.00</td><td style="text-align: right;">              2.86</td><td>Time Machine, The (2002)</td><td>Adventure</td></tr>
<tr><td style="text-align: right;">  3.4</td><td style="text-align: right;">3.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            3.00</td><td style="text-align: right;">              2.86</td><td>Time Machine, The (2002)</td><td>Action   </td></tr>
<tr><td style="text-align: right;">  3.3</td><td style="text-align: right;">3.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            3.00</td><td style="text-align: right;">              2.86</td><td>Time Machine, The (2002)</td><td>Sci-Fi   </td></tr>
<tr><td style="text-align: right;">  1.9</td><td style="text-align: right;">2.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.75</td><td style="text-align: right;">              3.52</td><td>Gangs of New York (2002)</td><td>Crime    </td></tr>
<tr><td style="text-align: right;">  1.9</td><td style="text-align: right;">1.5</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.75</td><td style="text-align: right;">              3.52</td><td>Road to Perdition (2002)</td><td>Crime    </td></tr>
<tr><td style="text-align: right;">  1.9</td><td style="text-align: right;">1.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.50</td><td style="text-align: right;">              4.00</td><td>Beautiful Mind, A (2001)</td><td>Drama    </td></tr>
<tr><td style="text-align: right;">  1.5</td><td style="text-align: right;">1.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.00</td><td style="text-align: right;">              4.00</td><td>Beautiful Mind, A (2001)</td><td>Romance  </td></tr>
<tr><td style="text-align: right;">  1.5</td><td style="text-align: right;">2.0</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.50</td><td style="text-align: right;">              3.52</td><td>Gangs of New York (2002)</td><td>Drama    </td></tr>
<tr><td style="text-align: right;">  1.5</td><td style="text-align: right;">1.5</td><td style="text-align: right;">    36</td><td style="text-align: right;">            1.50</td><td style="text-align: right;">              3.52</td><td>Road to Perdition (2002)</td><td>Drama    </td></tr>
</tbody>
</table>

In [30]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    d = np.square(a - b)
    return np.sum(d)

In [31]:
# Public tests
test_sq_dist(sq_dist)

All tests passed!


In [32]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1)}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2)}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3)}")

squared distance between a1 and b1: 0.0
squared distance between a2 and b2: 0.030000000000000054
squared distance between a3 and b3: 2


In [33]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = Model(input_item_m, vm_m)
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_2 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 41376 (161.62 KB)
Trainable params: 41376 (161.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

59/59 [==============================] - 0s 1ms/step
size of all predicted movie feature vectors: (1883, 32)


In [ ]:
count = 50
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])

m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    genre1,_  = get_item_genre(item_vecs[i,:], ivs, item_features)
    genre2,_  = get_item_genre(item_vecs[min_idx,:], ivs, item_features)

    disp.append( [movie_dict[movie1_id]['title'], genre1,
                  movie_dict[movie2_id]['title'], genre2]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".2f"])
table

<table>
<thead>
<tr><th>movie1                                </th><th>genres   </th><th>movie2                                             </th><th>genres   </th></tr>
</thead>
<tbody>
<tr><td>Save the Last Dance (2001)            </td><td>Drama    </td><td>John Q (2002)                                      </td><td>Drama    </td></tr>
<tr><td>Save the Last Dance (2001)            </td><td>Romance  </td><td>Sweetest Thing, The (2002)                         </td><td>Romance  </td></tr>
<tr><td>Wedding Planner, The (2001)           </td><td>Comedy   </td><td>National Lampoon&#x27;s Van Wilder (2002)               </td><td>Comedy   </td></tr>
<tr><td>Wedding Planner, The (2001)           </td><td>Romance  </td><td>Mr. Deeds (2002)                                   </td><td>Romance  </td></tr>
<tr><td>Hannibal (2001)                       </td><td>Horror   </td><td>Final Destination 2 (2003)                         </td><td>Horror   </td></tr>
<tr><td>Hannibal (2001)                       </td><td>Thriller </td><td>Sum of All Fears, The (2002)                       </td><td>Thriller </td></tr>
<tr><td>Saving Silverman (Evil Woman) (2001)  </td><td>Comedy   </td><td>Cats &amp; Dogs (2001)                                 </td><td>Comedy   </td></tr>
<tr><td>Saving Silverman (Evil Woman) (2001)  </td><td>Romance  </td><td>Save the Last Dance (2001)                         </td><td>Romance  </td></tr>
<tr><td>Down to Earth (2001)                  </td><td>Comedy   </td><td>Joe Dirt (2001)                                    </td><td>Comedy   </td></tr>
<tr><td>Down to Earth (2001)                  </td><td>Fantasy  </td><td>Haunted Mansion, The (2003)                        </td><td>Fantasy  </td></tr>
<tr><td>Down to Earth (2001)                  </td><td>Romance  </td><td>Joe Dirt (2001)                                    </td><td>Romance  </td></tr>
<tr><td>Mexican, The (2001)                   </td><td>Action   </td><td>Knight&#x27;s Tale, A (2001)                            </td><td>Action   </td></tr>
<tr><td>Mexican, The (2001)                   </td><td>Comedy   </td><td>Knight&#x27;s Tale, A (2001)                            </td><td>Comedy   </td></tr>
<tr><td>15 Minutes (2001)                     </td><td>Thriller </td><td>Panic Room (2002)                                  </td><td>Thriller </td></tr>
<tr><td>Heartbreakers (2001)                  </td><td>Comedy   </td><td>Animal, The (2001)                                 </td><td>Comedy   </td></tr>
<tr><td>Heartbreakers (2001)                  </td><td>Crime    </td><td>Stepford Wives, The (2004)                         </td><td>Thriller </td></tr>
<tr><td>Heartbreakers (2001)                  </td><td>Romance  </td><td>Bewitched (2005)                                   </td><td>Romance  </td></tr>
<tr><td>Spy Kids (2001)                       </td><td>Action   </td><td>Lara Croft: Tomb Raider (2001)                     </td><td>Action   </td></tr>
<tr><td>Spy Kids (2001)                       </td><td>Adventure</td><td>Lara Croft: Tomb Raider (2001)                     </td><td>Adventure</td></tr>
<tr><td>Spy Kids (2001)                       </td><td>Children </td><td>Princess Diaries, The (2001)                       </td><td>Children </td></tr>
<tr><td>Spy Kids (2001)                       </td><td>Comedy   </td><td>Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (2002)</td><td>Comedy   </td></tr>
<tr><td>Along Came a Spider (2001)            </td><td>Action   </td><td>Swordfish (2001)                                   </td><td>Action   </td></tr>
<tr><td>Along Came a Spider (2001)            </td><td>Crime    </td><td>Swordfish (2001)                                   </td><td>Crime    </td></tr>
<tr><td>Along Came a Spider (2001)            </td><td>Mystery  </td><td>Ring, The (2002)                                   </td><td>Mystery  </td></tr>
<tr><td>Along Came a Spider (2001)            </td><td>Thriller </td><td>Signs (2002)                                       </td><td>Thriller </td></tr>
<tr><td>Blow (2001)                           </td><td>Crime    </td><td>Training Day (2001)                                </td><td>Crime    </td></tr>
<tr><td>Blow (2001)                           </td><td>Drama    </td><td>Training Day (2001)                                </td><td>Drama    </td></tr>
<tr><td>Bridget Jones&#x27;s Diary (2001)          </td><td>Comedy   </td><td>Super Troopers (2001)                              </td><td>Comedy   </td></tr>
<tr><td>Bridget Jones&#x27;s Diary (2001)          </td><td>Drama    </td><td>Others, The (2001)                                 </td><td>Drama    </td></tr>
<tr><td>Bridget Jones&#x27;s Diary (2001)          </td><td>Romance  </td><td>Punch-Drunk Love (2002)                            </td><td>Romance  </td></tr>
<tr><td>Joe Dirt (2001)                       </td><td>Adventure</td><td>Daredevil (2003)                                   </td><td>Action   </td></tr>
<tr><td>Joe Dirt (2001)                       </td><td>Comedy   </td><td>Dr. Dolittle 2 (2001)                              </td><td>Comedy   </td></tr>
<tr><td>Joe Dirt (2001)                       </td><td>Mystery  </td><td>Doom (2005)                                        </td><td>Horror   </td></tr>
<tr><td>Joe Dirt (2001)                       </td><td>Romance  </td><td>Down to Earth (2001)                               </td><td>Romance  </td></tr>
<tr><td>Crocodile Dundee in Los Angeles (2001)</td><td>Comedy   </td><td>Heartbreakers (2001)                               </td><td>Comedy   </td></tr>
<tr><td>Crocodile Dundee in Los Angeles (2001)</td><td>Drama    </td><td>Scary Movie 4 (2006)                               </td><td>Horror   </td></tr>
<tr><td>Mummy Returns, The (2001)             </td><td>Action   </td><td>Swordfish (2001)                                   </td><td>Action   </td></tr>
<tr><td>Mummy Returns, The (2001)             </td><td>Adventure</td><td>Rundown, The (2003)                                </td><td>Adventure</td></tr>
<tr><td>Mummy Returns, The (2001)             </td><td>Comedy   </td><td>American Pie 2 (2001)                              </td><td>Comedy   </td></tr>
<tr><td>Mummy Returns, The (2001)             </td><td>Thriller </td><td>Star Trek: Nemesis (2002)                          </td><td>Thriller </td></tr>
<tr><td>Knight&#x27;s Tale, A (2001)               </td><td>Action   </td><td>Mexican, The (2001)                                </td><td>Action   </td></tr>
<tr><td>Knight&#x27;s Tale, A (2001)               </td><td>Comedy   </td><td>Mexican, The (2001)                                </td><td>Comedy   </td></tr>
<tr><td>Knight&#x27;s Tale, A (2001)               </td><td>Romance  </td><td>Monster&#x27;s Ball (2001)                              </td><td>Romance  </td></tr>
<tr><td>Shrek (2001)                          </td><td>Adventure</td><td>Monsters, Inc. (2001)                              </td><td>Adventure</td></tr>
<tr><td>Shrek (2001)                          </td><td>Animation</td><td>Monsters, Inc. (2001)                              </td><td>Animation</td></tr>
<tr><td>Shrek (2001)                          </td><td>Children </td><td>Monsters, Inc. (2001)                              </td><td>Children </td></tr>
<tr><td>Shrek (2001)                          </td><td>Comedy   </td><td>Monsters, Inc. (2001)                              </td><td>Comedy   </td></tr>
<tr><td>Shrek (2001)                          </td><td>Fantasy  </td><td>Monsters, Inc. (2001)                              </td><td>Fantasy  </td></tr>
<tr><td>Shrek (2001)                          </td><td>Romance  </td><td>Monsoon Wedding (2001)                             </td><td>Romance  </td></tr>
<tr><td>Animal, The (2001)                    </td><td>Comedy   </td><td>Heartbreakers (2001)                               </td><td>Comedy   </td></tr>
</tbody>
</table>